<p style="font-size:32px; font-weight: bolder; text-align: center"> Colored-noise methods </p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

This notebook provides a hands-on counterpart to the "Colored-noise methods" lecture for the MOOC "Path Integrals in Atomistic Modeling". If you haven't done so already, check the [getting started](0-getting_started.ipynb) notebook to make sure that the software infrastructure is up and running. 

The different sections in this notebook match the parts this lecture is divided into:

1. [Generalized Langevin Equations](#gle)
2. [Equilibrium GLE sampling](#equilibrium)
3. [Non-Equilibrium GLE sanpling](#non-equilibrium)
4. [Combining GLE and PIMD](#pi-gle)
5. [Dynamical properties](#dynamics)

<p style="color:blue; font-weight:bold"> Questions in blue invite you to reflect on the results you are obtaining. If you are doing these notebooks as part of a course, there might be questions to answer relative to those parts. </p>

_NB: you should run these sections in order, as the later ones re-use some of the data and the definitions from the earlier ones. If you cannot do the full notebook in a single session, re-running the full notebook should take less than a minute, as long as you leave the outputs of the external i-PI calculations in place._

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import ase, ase.io
from ase.ga.utilities import get_rdf
import chemiscope
import pimdmooc
pimdmooc.add_ipi_paths()

<a id="gle"> </a>

# Generalized Langevin Equations 

This section provides a brief demonstration of the possibility of computing properties of a generalized Langevin equation written in the extended-phase-space form, as a function of the drift matrix $\mathbf{A}_p$ and the diffusion matrix $\mathbf{B}_p$ (the latter being usually fixed by a fluctuation-dissipation relation.

A matrix $\mathbf{M}_p$ coupling physical momentum $p$ and extended momenta $\mathbf{s}$ can be written down as a combination of blocks, following the convention

$$
\begin{array}{c|cc}
      &    p   &   \mathbf{s} \\ \hline
   p  & m_{pp} &  \mathbf{m}_p^T \\
\mathbf{s} & \bar{\mathbf{m}}_p &  \mathbf{M}\\
\end{array}
$$

In [ ]:
# this splits up a Mp matrix into the four blocks describing interactions between p and s
def gle_split(Mp):
    """ Splits a matrix in the various blocks """
    return Mp[:1,:1], Mp[:1, 1:], Mp[1:, :1], Mp[1:,1:]

The simplest property that can be computed from the GLE matrices are the memory kernels of the associated non-Markovian formulation. The friction kernel reads

$$
K(t) = a_{pp} \delta(t) - \mathbf{a}_p^T e^{-t \mathbf{A}} \bar{\mathbf{a}}_p
$$

and its Fourier transform 

$$
K(\omega) = 2 a_{pp} -2 \mathbf{a}_p^T \frac{\mathbf{A}}{\mathbf{A}^2+\omega^2} \bar{\mathbf{a}}_p
$$

In [ ]:
def Kt(Ap, t, with_delta=False):
    app, ap, bap, A = gle_split(Ap)
    return (app[0,0] if with_delta and t==0 else 0) - (ap@sp.linalg.expm(-A*t)@bap)[0,0]

def Kw(Ap, w):
    app, ap, bap, A = gle_split(Ap)
    return (2*app  - 2*(ap@A)@np.linalg.solve(A@A+np.eye(len(A))*w**2,bap))[0,0]

## Memory kernels

Different analytical forms of memory kernel can be obtained with appropriate parameterizations of the drift matrix. The ones given below yield $K(t)$ that are exponential, $K(t)=\gamma e^{-t/\tau}$ or different types of Dirac-$\delta$-like functions, that give a peaked memory kernel in the frequency domain (the corresponding functional form is rather cumbersome, you can find a thorough discussion [here](https://www.research-collection.ethz.ch/bitstream/handle/20.500.11850/152344/eth-2145-02.pdf)) - the functional form has been slightly modified to give more intuitive link between the parameters and the shape of $K(\omega)$. In all these cases, the parameter `app`, corresponding to $a_{pp}$, introduces an explicit Markovian term in the kernel.

In [ ]:
def Ap_exp(gamma, tau, app=0):
    """ Drift matrix for an exponential memory kernel.
    gamma: intensity of the friction
    tau: time scale of the exponential decay
    app: Markovian term
    """
    return np.asarray( [ [app,-np.sqrt(gamma/tau) ], [np.sqrt(gamma/tau), 1/tau ]])
def Ap_delta(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel.  
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app, np.sqrt(gamma*domega/2), np.sqrt(gamma*domega/2) ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [-np.sqrt(gamma*domega/2), -omega0, domega ]
                       ])
def Ap_delta_alt(gamma, omega0, domega, app=0):
    """ Drift matrix for a delta-like memory kernel. Alternative form with K(0)=0. 
    gamma: intensity of the friction
    omega0: frequency-domain center of the K(w) peak
    domega: width of the peak
    app: Markovian term
    """
    return np.asarray( [ [app,np.sqrt(gamma*domega/2), 0 ], 
                         [-np.sqrt(gamma*domega/2),  domega, omega0 ],
                         [0, -omega0, 0 ]
                       ])

Below you can plot $K(\omega)$ for the three functional forms above. Play around with the parameters to verify their effect on the shape of the memory kernel spectrum. 

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
# defaults: Ap_delta_alt(1, 1, 1, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1, 1, 0.1, 1e-8), w) for w in wgrid], 'r-' )  
# defaults: Ap_delta(1, 0.1, 0.01, 1e-8)
ax.loglog(wgrid, [Kw(Ap_delta(1, 0.1, 0.01, 1e-8), w) for w in wgrid], 'b-' ) 
# defaults: Ap_exp(1, 1, 0)
ax.loglog(wgrid, [Kw(Ap_exp(1, 1, 0), w) for w in wgrid], 'k-' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 

An important idea that makes it easy to reuse GLE parameters for different systems is that it is possible to "translate" the shape of $K(\omega)$ by scaling it by a factor $\alpha$. This is essentially a change of units, so scaling the kernel moves the curve right and up in the $(\omega, K)$ plane

In [ ]:
wgrid = np.geomspace(1e-3,1e3,200)
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r-' )
ax.loglog(wgrid, [Kw(0.1*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r:' )
ax.loglog(wgrid, [Kw(10*Ap_delta_alt(1,1,1,1e-4), w) for w in wgrid], 'r--' )
ax.set_xlabel(r"$\omega$ / a.u."); ax.set_ylabel(r"$K(\omega)$ / a.u."); 

<p style="color:blue; font-weight:bold">
In the case of the analytical memory functions above, you can actually also mimic the effect of scaling by changing the value of the parameters. What parameters should you use for Ap_delta below, so that the blue curve becomes identical to the red curve?
</p>

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.loglog(wgrid, [Kw(0.1*Ap_delta(1,1,0.1,1e-4), w) for w in wgrid], 'r--' )

# modify the parameters below ↓ ↓ ↓ ↓, corresponding to gamma, omega0, domega, app
ax.loglog(wgrid, [Kw(Ap_delta(1,1,1,0), w) for w in wgrid], 'b:' )

## GLE integration

We now see how a GLE can be integrated in its Markovian, extended-phase-space form. The idea is very similar to the integrator for Langevin dynamics: a free-particle propagator that propagates $(p, \mathbf{s})$ for a finite time step $dt$ without an external potential is combined with a velocity-Verlet integrator for the Hamiltonian part of the equations of motion. The GLE integrator can be formulated as 

$$
(p, \mathbf{s})^T \leftarrow \mathbf{T}_p (p, \mathbf{s})^T + \mathbf{S}_p \boldsymbol{\xi}^T
$$

where $\boldsymbol{\xi}$ is a vector of uncorrelated random numbers, $\mathbf{T}_p = e^{-\mathbf{A}_p dt}$, and $\mathbf{S}_p\mathbf{S}_p^T = \mathbf{C}_p - \mathbf{T}_p \mathbf{C}_p \mathbf{T}_p^T$. 

In [ ]:
# Example classes for VV and GLE integration. Should be rather self-explanatory. 
# We consider a particle with unit mass
class VVIntegrator(object):
    """ Velocity-Verlet integrator """
    def __init__(self, force, dt, q):
        """ force is a function that takes a vector of positions q and returns -dV/dq """
        self.force = force
        self.dt = dt
        self.f = force(q)
    
    def step(self, q, p):        
        p[:] += self.f * self.dt *0.5
        q[:] += p * self.dt
        self.f = self.force(q)
        p[:] += self.f * self.dt*0.5            
        
class GLEIntegrator(object):
    """ Finite time-step GLE integrator for a free particle """
    def __init__(self, Ap, Cp, dt):
        self.ns = len(Ap)-1
        self.Ap = Ap
        self.Cp = Cp
        self.dt = dt
        self.T = sp.linalg.expm(-Ap*self.dt*0.5)
        self.S = sp.linalg.cholesky(self.Cp - self.T @ self.Cp @self.T.T).T
    
    def step(self, p, s):        
        ps = np.vstack([p,s])
        # stores the "GLE force" contribution for analysis
        self._rf = self.T @ ps - ps + self.S @ np.random.normal(size=(self.ns+1, len(p)))
        ps += self._rf        
        p[:] = ps[0]
        s[:] = ps[1:]

We can then run a trajectory for a free particle, using an exponential memory kernel. We run a 2D trajectory, but the two directions are equivalent

In [ ]:
# initialize the trajectory
q = np.asarray([0.,0.])
p = np.asarray([0.,0.])
s = np.asarray([[0.,0.]])
dt = 0.1
Ap = Ap_exp(0.2, 10, 0.01)  # default value: Ap_exp(0.2, 10, 0.01)
GLE = GLEIntegrator(Ap, np.eye(s.shape[0]+1), dt)
VV = VVIntegrator(lambda x:0.*x, dt, q)

In [ ]:
nstep = 40000 # default value: 40000
traj_q = np.zeros((nstep, len(q)))
traj_p = np.zeros((nstep, len(p)))
traj_f = np.zeros((nstep, len(p)))
for istep in range(nstep):
    traj_q[istep] = q; traj_p[istep] = p
    GLE.step(p,s)
    traj_f[istep] = GLE._rf[0]/(0.5*dt) # this is \dot{p} + V'(q), see the exercise in the lecture notes
    VV.step(q,p)
    GLE.step(p,s)    

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(4,4), constrained_layout=True)
ax.plot(traj_q[:,0], traj_q[:,1], 'r-')
ax.set_xlabel("$q_1$"); ax.set_ylabel("$q_2$");

The trajectory of the particle is quite different from what you would get with a white-noise Langevin run. Experiment with different parameters and types of the `Ap` matrix. 
<p style="color:blue; font-weight:bold">
Run a white-noise simulation with a_pp = 0.2 (you can achieve this by manually constructing an Ap matrix, or by setting gamma=0 in the exponential kernel). Observe the difference in the trajectory behavior: can you recognize this out of several trajectories generated with colored noise?
</p>

<a id="equilibrium"> </a>

# Equilibrium GLE sampling

This section requires use of i-PI, so make sure you have it installed and have familiarized yourself with how to run it in the [getting started](0-getting_started.ipynb) section. 

We will set up and run a molecular dynamics simulation of liquid water using a "smart-sampling" GLE thermostat, using the parameter generator from the [GLE4MD website](https://gle4md.org). Given that we will compare the sampling efficiency to that obtained from white-noise Langevin simulations, you may want to run the exercises from section 4 of the [sampling and MD notebook](1-md_sampling.ipynb). 

For reference, these are the values of the correlation times for $V$ and $K$ obtained from a long simulation with different white noise thermostat relaxation time $\tau=1/\gamma$


|  $\tau$ / fs   |    $\tau_V$ / ps  |   $\tau_K$ / ps   |
|----------------|-------------------|-------------------|
|     1          |        10         |      0.0009       |
|    100         |        0.8        |      0.04         |
|   10000        |        4.6        |      1.5          |

We also load some reference data that was generated with those trajectories, that contains the correlation functions of potential and kinetic energy at different Langevin $\tau$ (format: `[time{ps}, acf_V(tau=1fs),   acf_K(tau=1fs), acf_V(tau=100fs), acf_K(tau=100fs), acf_V(tau=10ps), acf_K(tau=10ps)]`)

In [ ]:
ref_data = np.loadtxt('5-gle/ref_langevin_acf.dat')

Now we can prepare the GLE trajectory. Make a copy of the template file

```
$ cd pimd-mooc/5-gle
$ cp template_gle.xml input.xml
```

and modify it to use a GLE thermostat. We will use a "smart-sampling" GLE, so set the prefix to `md-gle_smart`, to match the post-processing below.

Then, the important part: setting the GLE parameters. We will use the on-line generator on the [GLE4MD](https://gle4md.org) website. The website does not fit parameters from scratch, but uses a library of pre-optimized parameters, and uses scaling rules to adjust them for the range of frequencies of interest. 

![the input generation interface on gle4md.org](figures/gle4md-screenshot.png)

"Smart" GLE thermostats are designed to provide optimal sampling efficiency for a characteristic time scale. We set it to 5 ps, given that the example is limited to 20ps and so 5-10ps is the longest time scale we can hope to target. By choosing a parameters preset that targets 3 orders of magnitude in frequency, we get "as efficient as possible" sampling up to about 6000 cm<sup>-1</sup>, well above the highest vibrational frequencies in water that are around 3600 cm<sup>-1</sup>. We set the formatting to i-PI output, that generates an XML block that should be copied and pasted within the `<dynamics>` block in the input file. 
You can get parameters that are suitable for water following [this link](https://gle4md.org/index.html?page=matrix&kind=smart&tslow=5&utslow=ps&smrange=6-3&outmode=ipi)

Having set up the input file, you can run it as usual, launching i-PI first, and then the driver code that computes q-TIP4P/f energies and forces. 

```
$ i-pi input.xml &> log &
$ i-pi-driver -u -h driver -m qtip4pf 
```

Wait until the run has completed, then load the output trajectory and continue the analysis

In [ ]:
traj_gle = pimdmooc.read_ipi_output('5-gle/md-gle_smart.out')
pos_gle = pimdmooc.read_ipi_xyz('5-gle/md-gle_smart.pos_0.xyz')

We get the radial distribution functions, to get an idea of the structure of the liquid. These will also be used further down to compare with quantum simulations.
_NB: ASE normalizes partial RDF in such a way they do not tend to 1 for a homogeneous system. We correct manually the normalization_

In [ ]:
rbins = get_rdf(pos_gle[0], rmax=4.5, nbins=200, elements=[8, 8])[1] 
rdf_cls_oo = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 8])[0] for f in pos_gle[::10]]).mean(axis=0)/(1/3)
rdf_cls_oh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 1])[0] for f in pos_gle[::10]]).mean(axis=0)/(2/3)
rdf_cls_hh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[1, 1])[0] for f in pos_gle[::10]]).mean(axis=0)/(2/3)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(rbins, rdf_cls_oo, 'r-' )
ax.plot(rbins, rdf_cls_oh, c='gray' )
ax.plot(rbins, rdf_cls_hh, 'c-' )
ax.set_xlabel(r"$r / \AA$"); ax.set_ylabel(r"RDF"); 
ax.set_ylim(-0.1,5)

We compute the autocorrelation function of potential and kinetic energy for the trajectory. 

In [ ]:
acf_v_gle = pimdmooc.autocorrelate(traj_gle["potential"], normalize=True)
acf_k_gle = pimdmooc.autocorrelate(traj_gle["kinetic_md"], normalize=True)

In [ ]:
# integral-by-sum (we truncate at ~5ps because of the high level of noise)
tau_v = (acf_v_gle[:5000].sum() - 0.5*acf_v_gle[0])*traj_gle["time"][1]
tau_k = (acf_k_gle[:5000].sum() - 0.5*acf_k_gle[0])*traj_gle["time"][1]

In [ ]:
print("Autocorrelation time: tau_V = % 10.5f ps,  tau_K = % 10.5f ps" % (tau_v, tau_k))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3.5))
acf_len = 10000
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,2], color=(0.5,0,0,0.5), label=r"$K, \tau=1$ fs")
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,4], color=(1,0,0,0.5), label=r"$K, \tau=100$ fs")
ax[0].plot(ref_data[:acf_len,0], ref_data[:acf_len,6], color=(1,0.5,0.5,0.5), label=r"$K, \tau=10$ ps")
ax[0].plot(ref_data[:acf_len,0], acf_k_gle[:acf_len], color='k', label=r"$K, $ GLE")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,1], color=(0,0,0.5,0.5), label=r"$V, \tau=1$ fs")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,3], color=(0,0,1,0.5), label=r"$V, \tau=100$ fs")
ax[1].plot(ref_data[:acf_len,0], ref_data[:acf_len,5], color=(0.5,0.5,1,0.5), label=r"$V, \tau=10$ ps")
ax[1].plot(ref_data[:acf_len,0], acf_v_gle[:acf_len], color='k', label=r"$V, $ GLE")
for a in ax:
    a.legend(ncol=1)
    a.legend(ncol=1)
    a.set_xlabel("time / ps"); a.set_ylabel("energy / a.u.");

<p style="color:blue; font-weight:bold">
Observe the autocorrelation functions, and compare them with those obtained by white-noise thermostats. 
</p>
<em>NB: the reference trajectories are obtained from much longer simulations, so they have smaller error in the asymptotic regime. Those from the GLE run will be noisier so you have to use a bit of "mental filtering". You can set up the GLE run to be longer, if you can afford the wait!</em>

<p style="color:blue; font-weight:bold">
Imagine what would happen if you set the masses of all the atoms to be 100 times larger. What would you change in the setup of the simulation, what would change in the results and what will not?
</p>

<a id="non-equilibrium"> </a>

# Non-equilibrium GLE sampling

Let's now see what happens when using a GLE simulation that does not fulfill the classical fluctuation-dissipation theorem, $k_B T(\mathbf{A}_p+\mathbf{A}_p^T)\ne \mathbf{B}_p \mathbf{B}_p^T$. 
In fact, the GLE still reaches (in the free-particle limit) a stationary state for which the covariance matrix $\mathbf{C}_p$ is not diagonal, and is consistent with $\mathbf{A}_p \mathbf{C}_p+\mathbf{C}_p\mathbf{A}_p^T = \mathbf{B}_p \mathbf{B}_p^T$: in practice, one usually considers $\mathbf{C}_p$ as the GLE parameter, given also that $\mathbf{B}_p$ is not needed to compute the finite-time propagator. 

We demonstrate the use of a _quantum thermostat_, in which $\mathbf{A}_p$ and $\mathbf{C}_p$ are optimized to yield a distribution of $\langle q\rangle^2$ and  $\langle p\rangle^2$ consistent with the quantum expectation values for a harmonic oscillator of frequency $\omega_0$. 

Similar to the equilibrium case, several pre-optimized GLE parameters (that also balance sampling efficiency and zero-point energy leakage) can be obtained from the [GLE4MD website](https://gle4md.org). Scaling rules make it possible to adjust the target temperature, and the most important parameter is the maximum frequency for which the quantum fluctuations are guaranteed to be correct in the harmonic limit. 

## Quantum thermostat for a harmonic oscillator

As a demonstration, we run a simulation for a 3D harmonic oscillator. Everything is expressed in atomic units, the target temperature is $T=1$ and the frequencies are $\omega_0 = 0.25, 1, 4$: the lowest frequency is in the classical regime, the intermediate one is at the turning point between classical and quantum, and the highest frequency is strongly quantized.  Given that for the highest frequency $\hbar\omega\beta=4$, the presets which fits fluctuations up to $\hbar\omega\beta=20$ is more than sufficient. 
The GLE parameters below are obtained from the [online GLE input generator](https://gle4md.org/index.html?page=matrix&kind=quantum&parset=20_6&temp=1&utemp=aue&outmode=python&aunits=aut&cunits=aue)

_NB: this uses the subroutines defined in [section 1](#gle)._

In [ ]:
# initialize the trajectory
q = np.asarray([0.,0.,0.])
p = np.asarray([0.,0.,0.])
s = np.zeros(shape=(6,3))
dt = 0.1  # default: 0.1
# strong bhw=20 quantum thermostat matrices for beta=1 a.u., from the GLE4MD website
Ap = np.asarray([
[   9.940227881069e-3,    1.191832744031e+0,    7.841346537680e-1,    1.127422061083e+0,    1.287760047739e+0,    6.597371849521e-1,    3.854520538662e-1, ],
[  -1.219402549722e+0,    5.187757030411e-1,    1.679849599124e+0,   -2.171362088679e-1,   -5.679884059178e-2,    1.678648983902e-1,   -1.694069965777e+0, ],
[  -7.396592199758e-1,   -1.679849599124e+0,    5.313365730649e-1,    2.916457167952e-1,    7.922023001118e-1,    2.804659293960e-1,   -8.312829730079e-1, ],
[  -1.129202488515e+0,    2.171362088679e-1,   -2.916457167952e-1,    6.075528350225e-1,    2.238529963876e-2,    7.625335027833e-1,    3.896382327408e-1, ],
[  -1.251885757500e+0,    5.679884059178e-2,   -7.922023001118e-1,   -2.238529963876e-2,    6.636392814216e-1,   -8.806083934866e-1,    2.480987428195e+0, ],
[  -3.314567459043e-1,   -1.678648983902e-1,   -2.804659293960e-1,   -7.625335027833e-1,    8.806083934866e-1,    6.023159253293e+0,   -8.283882517564e+0, ],
[  -7.647113842221e-1,    1.694069965777e+0,    8.312829730079e-1,   -3.896382327408e-1,   -2.480987428195e+0,    8.283882517564e+0,    9.760847161873e+0, ],
])
Cp = np.asarray([
[   9.999953047000e-1,    1.979197779000e-2,    7.147922505000e-1,    1.961018636000e-1,   -3.732679220000e-1,   -2.264460588000e-1,    4.599299108000e-2, ],
[   1.979197779000e-2,    1.260617101000e+0,   -1.931506174000e-1,   -7.262575605000e-1,   -5.497702197000e-1,   -2.185704484000e-1,    1.277943581000e-1, ],
[   7.147922505000e-1,   -1.931506174000e-1,    2.441919995000e+0,    7.295025710000e-1,   -1.234862177000e+0,   -1.128397955000e-1,   -1.609235586000e-2, ],
[   1.961018636000e-1,   -7.262575605000e-1,    7.295025710000e-1,    2.320143840000e+0,   -1.210057031000e+0,    5.676469950000e-1,   -2.024421968000e-1, ],
[  -3.732679220000e-1,   -5.497702197000e-1,   -1.234862177000e+0,   -1.210057031000e+0,    4.792907650000e+0,    3.749158013000e-1,    1.177918093000e-2, ],
[  -2.264460588000e-1,   -2.185704484000e-1,   -1.128397955000e-1,    5.676469950000e-1,    3.749158013000e-1,    4.964464101000e+0,   -5.358672711000e-1, ],
[   4.599299108000e-2,    1.277943581000e-1,   -1.609235586000e-2,   -2.024421968000e-1,    1.177918093000e-2,   -5.358672711000e-1,    1.481694550000e+0, ],
]
)

GLE = GLEIntegrator(Ap, Cp, dt)
omega = np.asarray([0.25,1,4])   # default: [0.25,1,4]
VV = VVIntegrator(lambda x: -x*omega**2, dt, q) # harmonic force as a lambda function ^_^

In [ ]:
nstep = 200000 # default value: 200000
traj_q = np.zeros((nstep, len(q)))
traj_p = np.zeros((nstep, len(p)))
for istep in range(nstep):
    traj_q[istep] = q; traj_p[istep] = p
    GLE.step(p,s)
    VV.step(q,p)
    GLE.step(p,s)    

... do you recognize the expressions for the distribution of $q$ for a harmonic oscillator?

In [ ]:
def cho_pq(w, beta, q):
    """ Distribution of q for a classical harmonic oscillator. """
    q2 = beta/w**2
    return np.exp(-0.5*q**2/q2)/np.sqrt(2*np.pi*q2)
def qho_pq(w, beta, q):
    """ Distribution of q for a quantum harmonic oscillator. """
    q2 = 0.5*beta/w /np.tanh(beta*w/2)
    return np.exp(-0.5*q**2/q2)/np.sqrt(2*np.pi*q2)

The distribution of positions sampled by the quantum thermostat follows closely the _quantum_ distribution, as it can be seen by plotting the histogram of positions (note that statistical convergence might be a problem, you can try increase the duration of the simulation to obtain more precise convergence). 

In [ ]:
qgrid = np.linspace(-15,15,1000)
fig, ax = plt.subplots(1,3,figsize=(11,3), constrained_layout=True)
for i in range(3):
    ax[i].hist(traj_q[:,i], bins=100, density=True, label="quantum thermostat", color="gray");
    ax[i].plot(qgrid, qho_pq(omega[i], 1, qgrid), 'r-', label="quantum HO")
    ax[i].plot(qgrid, cho_pq(omega[i], 1, qgrid), 'b--', label="classical HO")
    ax[i].set_xlim(-5/omega[i], 5/omega[i])
    ax[i].set_title(r"$\omega=$"+str(omega[i]))
    ax[i].set_xlabel(r"$q$ / a.u.")
ax[2].legend()

Try to run with higher frequencies. Is the simulation still accurate when $\omega>20$? 
_NB: as you increase the oscillator frequency you may have to reduce the time step._

## Quantum thermostat for liquid water

Now modify the `template_gle.xml` input file (after copying it to a different name) to perform a quantum thermostat simulation for water at 300K. It is recommended that you use `prefix='md-gle_qt'`, to be consistent with the post-processing code below.
Try to choose the parameters on [GLE4MD website](https://gle4md.org/index.html?page=matrix). Strongly coupled matrices that guarantee quantum behavior up to a cutoff frequency around 4000 cm<sup>-1</sup> should be suitable. 
If you are uncertain or want to check your selection, compare it with [the recommended parameters](https://gle4md.org/index.html?page=matrix&kind=quantum&parset=20_6&temp=300&utemp=k&outmode=ipi).

After having run the simulations, we can look at the results

In [ ]:
traj_qt = pimdmooc.read_ipi_output('5-gle/md-gle_qt.out')
pos_qt = pimdmooc.read_ipi_xyz('5-gle/md-gle_qt.pos_0.xyz')

We print out separately the kinetic temperature of oxygen and hydrogen atoms. This is a striking indication of the non-equilibrium nature of the thermostat, if seen in a classical sense: the kinetic energy of H atoms differs dramatically from that of the O atoms, and the overall kinetic energy is much higher than that one would expect from classical equipartition at $T=300$ K.

_NB: let us reiterate that the temperature computed from the kinetic energy is *not* a valid proxy of the thermodynamic temperature in a quantum simulation, because the relationship between mean kinetic energy per degree of freedom and $T$ only holds in the classical case._

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(traj_qt["time"], traj_qt["temperature(O)"], 'r', ls="-" , label="O")
ax.plot(traj_qt["time"], traj_qt["temperature(H)"], 'c', ls="-", label="H" )
ax.plot(traj_qt["time"], traj_qt["temperature"], 'k', ls="-", label="all" )
ax.set_xlabel(r"$t$ / ps"); ax.set_ylabel(r"T / K"); 
ax.legend();

We can then look at the radial distribution functions.

In [ ]:
rbins = get_rdf(pos_qt[0], rmax=4.5, nbins=200, elements=[8, 8])[1] 
rdf_qt_oo = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 8])[0] for f in pos_qt[1000::10]]).mean(axis=0)/(1/3)
rdf_qt_oh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 1])[0] for f in pos_qt[1000::10]]).mean(axis=0)/(2/3)
rdf_qt_hh = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[1, 1])[0] for f in pos_qt[1000::10]]).mean(axis=0)/(2/3)

One sees that the long-range parts of the RDF, and most of the O-O pair distribution, are not affected by the quantum thermostat. This is very similar to the result of PIMD simulations with this potential. On the other hand, intra-molecular degrees of freedom, identified by the first peak in the O-H and H-H RDF, are much broader, reflecting the larger quantum fluctuations driven by the GLE, that simulates the effect of zero-point energy motion. 

In [ ]:
# note that this reuses the classical RDF from section 2
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(rbins, rdf_cls_oo, 'r', ls=":" )
ax.plot(rbins, rdf_cls_oh, c='gray', ls=":" )
ax.plot(rbins, rdf_cls_hh, 'c', ls=":" )
ax.plot(rbins, rdf_qt_oo, 'r', ls="-" )
ax.plot(rbins, rdf_qt_oh, c='gray', ls="-" )
ax.plot(rbins, rdf_qt_hh, 'c', ls="-" )
ax.set_xlabel(r"$r / \AA$"); ax.set_ylabel(r"RDF"); 
ax.set_ylim(-0.1,5)

<p style="color:blue; font-weight:bold">
Plot the potential and kinetic energy (or compute directly the mean), for both this trajectory and the classical GLE simulation in section 2. How much additional energy per water molecule is present in the system due to quantum fluctuations?
</p>

_NB: remember that i-PI returns energies in atomic units. one atomic unit of energy equals approximately 27.2 eV_

**Something more:** _Re-run these simulations using the "weak coupling" GLE. You should observe colder H, hotter O, and loss of structure in the long-range part of the RDF, that are all indications of zero-point energy leakage from high-frequency to low-frequency vibrations_

<a id="pi-gle"> </a>

# Combining GLE and PIMD

The quantum thermostat, demonstrated in the [previous section](#non-equilibrium), provides an inexpensive way to simulate quantum fluctuations in both harmonic and quasi-harmonic systems. In the presence of large anharmonicities, however, or for extremely low temperatures, this is only an approximation, and one that cannot be systematically tested or improved upon. One possibility is to use a GLE *on top of a PIMD simulation*, so as to be able to converge progressively the thermostat to equilibrium sampling in the ring-polymer phase space, as the number of replicas $P$ increases.

_NB: Path integral simulations, even with GLE acceleration, are rather time consuming. The default duration (10ps) is on the short side, so expect higher statistical errors than in other exercises. Even with short simulations, these will take more than one hour so we suggest you set up all simulations in parallel (e.g. in different terminal tabs), using different UNIX socket names, and let them run while you take a break._

In [ ]:
# we run with different # of beads, so we'll store the trajectory data in arrays
traj_pg = {}
pos_pg = {}
p_list = [2,4,6]

Start by setting up simulations using `template_piglet.xml` as a template. As usual, copy to a different file name before editing, to leave a clean template. Beside setting prefix and socket name to indicate the number of beads in the simulation, you will have to set two key parameters: the actual number of beads in the simulation, the `nbeads='xxx'` attribute in the `<initialize>` tag, and the actual GLE parameters. 

You can fetch those from [GLE4MD](https://gle4md.org) rather easily, although the settings panel is somewhat richer than for simpler GLE schemes:

![the input generation interface for PIGLET parameters](figures/gle4md-piglet.png)

Most importantly, you need to set the number of beads to match that in the simulation. Since PIGLET adds in normal-modes coordinates, the input you'll have to copy is rather cumbersome, with a separate set of GLE parameters for each normal mode of the ring polymer. Then, you need to set a separate thermostat for the centroid. The "optimal sampling" scheme is recommended, not only for the sampling efficiency but also because it effectively contrasts zero-point energy leakage - which is still an issue, although less so than in the case of the quantum thermostat. 
The other parameters should be self-explanatory. You can check your parameters against [this example](https://gle4md.org/index.html?page=matrix&kind=piglet&centroid=kh_8-4&cw0=4000&ucw0=cm1&nbeads=2&temp=300&utemp=k&parset=20_8_t&outmode=ipi) for the $P=2$ case. 

Set up and run simulations for $P=2, 4, 6$, then move on to the analysis.

In [ ]:
for p in p_list:
    traj_pg[p] = pimdmooc.read_ipi_output(f'5-gle/md-piglet_{p}.out')
    pos_pg[p] = pimdmooc.read_ipi_xyz(f'5-gle/md-piglet_{p}.pos_0.xyz')

First, let's look at the kinetic temperature of the simulation. **This number is completely meaningless** in physical terms, because of the complicated thermostatting scheme: the kinetic energy of the system is computed by the centroid-virial estimator, which we will use later. However it is useful to verify the non-equilibrium nature of the thermostat: for $P\rightarrow\infty$, where the path integral formalism alone should suffice to achieve quantum sampling, the estimator should converge to the target temperature of the simulation, T=300K. Plot the kinetic temperature of O and H atoms for the different numbers of beads, and see how they converge. Note also the very fast initial relaxation: this is because the run is initiated from a single configuration, and the beads rapidly spread out to the extent of ring polymer fluctuations. Full equilibration - particularly for structural properties - may take longer. 
_NB: this is a rather naive argument, and the convergence of PIGLET temperatures to 300K is rather slow due to the actual definitions of target temperatures. You should however notice the reduction in temperature difference between H and O, which is indicative of the reduced impact of zero-point energy leakage._

In [ ]:
p = 6
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.set_title(f"PIGLET simulations, P={p}")
ax.plot(traj_pg[p]["time"], traj_pg[p]["temperature(O)"], 'r', ls="-" , label="O")
ax.plot(traj_pg[p]["time"], traj_pg[p]["temperature(H)"], 'c', ls="-", label="H" )
ax.plot(traj_pg[p]["time"], traj_pg[p]["temperature"], 'k', ls="-", label="all" )
ax.set_xlabel(r"$t$ / ps"); ax.set_ylabel(r"T / K"); 
ax.legend();

You can also plot the actual potential and kinetic energy for the different runs

In [ ]:
p = 6
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.set_title(f"PIGLET simulations, P={p}")
ax.plot(traj_pg[p]["time"], traj_pg[p]["potential"], 'r', ls="-" , label="V")
ax.plot(traj_pg[p]["time"], traj_pg[p]["kinetic_cv"], 'b', ls="-", label="K" )
ax.set_xlabel(r"$t$ / ps"); ax.set_ylabel(r"energy / a.u."); 
ax.legend();

Computing the means demonstrate the fast quantitative convergence of the scheme to the quantum expectation values.

In [ ]:
mean_v = []
mean_k = []
mean_v_cls = -0.50735 # this is a reference value computed from a long classical run
mean_k_cls = (96*3 - 3)*0.5*300*3.1668116e-06 # exact classical kinetic energy
mean_v_qm = -2.065e-01 # reference from a long PIMD run with 16 beads and Suzuki-Chin
mean_k_qm = 4.324e-01
for p in p_list:
    mean_v.append(traj_pg[p]["potential"][1000:].mean())
    mean_k.append(traj_pg[p]["kinetic_cv"][1000:].mean())

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(p_list, mean_v, 'r', ls="-" , marker = 'o', label="V")
ax.plot(p_list, mean_k, 'b', ls="-", marker="*", label="K" )
ax.hlines(mean_v_cls, 2, 6, color='r', ls=":", label="V (cls)")
ax.hlines(mean_k_cls, 2, 6, color='b', ls=":", label="K (cls)")
ax.hlines(mean_v_qm, 2, 6, color='r', ls="--", label="V (qm)")
ax.hlines(mean_k_qm, 2, 6, color='b', ls="--", label="K (qm)")
ax.set_xlabel(r"$t$ / ps"); ax.set_ylabel(r"energy / a.u."); 
ax.legend();

We can also compute the RDF. Note the difficulty in converging these with a short simulation: you could get smoother curves by averaging over all beads (they are output so you only need to implement a nested sum) but the typical relaxation time for the O-O correlations is of the order of 10ps, and so longer simulations would really be needed. 

In [ ]:
# loads RDF references from a long-ish Suzuki-Chin run
_, rdf_sc_oo, rdf_sc_oh, rdf_sc_hh = np.loadtxt("5-gle/ref_rdf_sc.dat").T

In [ ]:
rbins = get_rdf(pos_pg[2][0], rmax=4.5, nbins=200, elements=[8, 8])[1]
rdf_pg_oo = {}; rdf_pg_oh = {}; rdf_pg_hh = {}
for p in [2,4,6]:
    rdf_pg_oo[p] = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 8])[0] for f in pos_pg[p][100::2]]).mean(axis=0)/(1/3)
    rdf_pg_oh[p] = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[8, 1])[0] for f in pos_pg[p][100::2]]).mean(axis=0)/(2/3)
    rdf_pg_hh[p] = np.asarray([ get_rdf(f, rmax=4.5, nbins=200, elements=[1, 1])[0] for f in pos_pg[p][100::2]]).mean(axis=0)/(2/3)

In [ ]:
p = 6
# note that this reuses the classical RDF from section 2
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(rbins, rdf_cls_oo, 'r', ls=":" )
ax.plot(rbins, rdf_cls_oh, c='gray', ls=":" )
ax.plot(rbins, rdf_cls_hh, 'c', ls=":" )
ax.plot(rbins, rdf_sc_oo, 'r', ls="--" )
ax.plot(rbins, rdf_sc_hh, 'c', ls="--" )
ax.plot(rbins, rdf_sc_oh, c='gray', ls="--" )
ax.plot(rbins, rdf_pg_oo[p], 'r', ls="-" )
ax.plot(rbins, rdf_pg_oh[p], c='gray', ls="-" )
ax.plot(rbins, rdf_pg_hh[p], c='c', ls="-" )
ax.set_xlabel(r"$r / \AA$"); ax.set_ylabel(r"RDF"); 
ax.set_ylim(-0.1,5)

<p style="color:blue; font-weight:bold">
Compute the quantum corrections to potential and kinetic energy per molecule. Also, plot the RDF of the PIGLET trajectories against those from the "quantum thermostat" simulations above. What can you conclude in terms of the shortcomings of QT simulations? You can also compare with the reference PIMD RDF to verify that PIGLET results are indeed closer to the correct quantum fluctuations.
</p>

_NB: you will need the results of [Section 3](#non-equilibrium), as well as the reference values defined above._

<a id="dynamics"> </a>

# Dynamical properties 

Let's look at the velocity-velocity correlation spectrum, that is the Fourier transform of the velocity-velocity correlation function. This has already been introduced in <a href="./3-rpmd.ipynb"> the RPMD module</a> as one of the key indicators of the dynamical properties of a system, being closely connected to the vibrational density of states, and a number of spectroscopic observables. 

Given that it involves the correlations between the velocities of many particles, we don't compute $c_{vv}$ manually from within this notebook, but use some of the tools that are provided with i-PI. If you have installed i-PI and set the path correctly, you should be able to run it from within the `pimd-mooc/5-gle` folder.
We use one of the outputs of the equilibrium GLE runs from [Section 2](#equilibrium), so make sure you have run that section and have not removed the outputs

```
$ i-pi-getacf -ifile md-gle_smart.vel_0.xyz -mlag 2000 -ftpad 500 -ftwin cosine-blackman \
              -dt "1.0 femtosecond" -oprefix cvv-gle_smart
```

_(either enter the command on a single line, or make sure you escape the carriage return)_. This will generate two files: `cvv-gle_smart_acf.data` contains the correlation function, and `cvv-gle_smart_facf.data` its Fourier transform.  We load it, together from a reference $c_{vv}$ from a reference calculation using a gentle global thermostat. 

In [ ]:
cvv_gle = np.loadtxt('5-gle/cvv-gle_smart_facf.data')
cvv_svr = np.loadtxt('5-gle/ref_cvv-svr_facf.data')

Even though the "smart-sampling" GLE is rather gentle, it clearly modifies the lineshape of the velocity correlation spectrum, broadening and slightly redshifting the bending and stretching peaks

In [ ]:
au2cm1 = 219474.63 # frequency axis is given in a.u.
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(au2cm1*cvv_gle[:,0], cvv_gle[:,1], 'r-')
ax.plot(au2cm1*cvv_svr[:,0], cvv_svr[:,1], 'gray', ls='--')
ax.set_xlim(0,5000); ax.set_xlabel(r"$\omega$ / cm$^{-1}$"); ax.set_ylabel(r"$c_{vv}$ / a.u.");

Now we can use another tool to apply the Richardson-Lucy deconvolution to the GLE-distorted spectrum, to recover the "true" dynamics. See [DOI: 10.1063/1.4990536](https://doi.org/10.1063/1.4990536) for a discussion of the underlying theory. 

The program needs the velocity correlation spectrum as well as the parameters of the GLE that was used to generate the underlying trajectory. It can extract them from the i-PI input file, if you have kept it

```
$ i-pi-gleacf -a deconv -ifacf cvv-gle_smart_facf.data -ixml smart.xml -ts 1.0 \
              -op cvv-gle_rl -s 2 -dp 100 10
```

or from a raw file that you can write containing the matrix [in the appropriate units and raw format](https://gle4md.org/index.html?page=matrix&kind=smart&tslow=5&utslow=ps&smrange=6-3&outmode=raw&aunits=aut)

```
$ i-pi-gleacf -a deconv -ifacf cvv-gle_smart_facf.data -ia smart_a -ts 1.0 \
              -op cvv-gle_rl -s 2 -dp 100 10
```

This generates a series of files `cvv-gle_rl_XXX.data`, that correspond to successive iterations of the R-L deconvolution. 

In [ ]:
cvv_rl = []
for i in range(10, 100, 10):
    cvv_rl.append(np.loadtxt('5-gle/cvv-gle_rl_%03d.data' % (i)) )

Note that iterating R-L too much amplifies noise in the data, so in practice one may want to implement an "early stop" criterion based on the balance between the residual error in the forward convolution and some measure of smoothness. 

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(au2cm1*cvv_gle[:,0], cvv_gle[:,1], 'r-')
ax.plot(au2cm1*cvv_rl[0][:,0], cvv_rl[0][:,1], c=(0.8,0,0.2))
ax.plot(au2cm1*cvv_rl[5][:,0], cvv_rl[5][:,1], c=(0.5,0,0.5))
ax.plot(au2cm1*cvv_rl[-1][:,0], cvv_rl[-1][:,1], c=(0.,0,1))
ax.plot(au2cm1*cvv_svr[:,0], cvv_svr[:,1], 'gray', ls='--')
ax.set_xlim(0,5000); ax.set_xlabel(r"$\omega$ / cm$^{-1}$"); ax.set_ylabel(r"$c_{vv}$ / a.u.");

We can also post-process PIGLET simulations to extract an estimate of the quantum time correlation functions.
If you haven't stored the XML input for the simulations in [Section 4](#pi-gle) you may have to re-create the one for $P=6$ before you run this command (that also needs the output of those simulations)

```
$ i-pi-gleacf -a deconv -ifacf cvv-piglet_6_facf.data -ixml piglet_6.xml -ts 1.0 \
              -op cvv-piglet_rl -s 2 -dp 500 10
```

In [ ]:
cvv_piglet = np.loadtxt('5-gle/cvv-piglet_6_facf.data')

In [ ]:
cvv_piglet_rl = []
for i in range(10, 500, 10):
    cvv_piglet_rl.append(np.loadtxt('5-gle/cvv-piglet_rl_%03d.data' % (i)) )

Note that given the short trajectories and the very aggressive thermostatting associated with the "OPT-H" centroid thermostat, it is hard to find a good balance between the level of noise and the progress of deconvolution. Still, one can see clearly the red-shift of the stretch peaks which is one of the hallmarks of quantum dynamical effects in liquid water.

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3), constrained_layout=True)
ax.plot(au2cm1*cvv_piglet[:,0], cvv_piglet[:,1], 'r-')
ax.plot(au2cm1*cvv_piglet_rl[0][:,0], cvv_piglet_rl[0][:,1], c=(0.8,0,0.2))
ax.plot(au2cm1*cvv_piglet_rl[5][:,0], cvv_piglet_rl[5][:,1], c=(0.5,0,0.5))
ax.plot(au2cm1*cvv_piglet_rl[-1][:,0], cvv_piglet_rl[-1][:,1], c=(0.,0,1))
ax.plot(au2cm1*cvv_svr[:,0], cvv_svr[:,1], 'gray', ls='--')
ax.set_xlim(0,5000); ax.set_xlabel(r"$\omega$ / cm$^{-1}$"); ax.set_ylabel(r"$c_{vv}$ / a.u.");

**Something more:** _You can try to run a longer simulation with the weaker "OPT-V" centroid thermostat, obtaining a better-converged and less dramatically broadened initial spectrum. This should give a more stable deconvolution, and a quantum spectrum in very good agreement with more expensive approximate quantum dynamics techniques_